<a href="https://colab.research.google.com/github/erikaminarikova/Ordinal_Sums/blob/main/Example_3_calculate_ordinal_sums_with_multiple_attributes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import statistics

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
from matplotlib import image

In [4]:
#ordina sums functions

# defining the generic functions for individual classes
def yes_function(x: float, y: float, a: float, kappa: float, lam: float, gamma: float = 1) -> float:
    return 1-max(0.0, (((1-x) ** kappa) + ((1-y) ** kappa) - ((1-a) ** kappa))) ** (1 / kappa)

def no_function(x: float, y: float, a: float, kappa: float, lam: float, gamma: float = 1) -> float:
        return max(0.0, ((x ** lam) + (y ** lam) - (a ** lam))) ** (1 / lam)

def maybe_function(x: float, y: float, a:float, kappa: float, lam: float, gamma: float = 1) -> float:
    if gamma == 0:
            # r = 0 means we use the geometric mean
        return 0
    elif gamma < 0:
            # r < 0 means we handle cases where x or y are 0
        if x == 0 or y == 0:
            return 0
        else:
                value = gamma*1/a*x*y+(1-gamma)*(1-1/(1-a)*(1-x)*(1-y))

    else:
            value = gamma*1/a*x*y+(1-gamma)*(1-1/(1-a)*(1-x)*(1-y))

    return statistics.median([0, 1, value])

# defining function for classification: deviding values according thir value to which fuction we use
def classification(x: float, y: float, a: float, kappa: float, lam: float, gamma: float) -> float:
    if x<a and y<a:
        value = no_function(x,y,a,1,lam,1)
    elif x>a and y>a:
        value = yes_function(x,y,a,kappa,1,1)
    else:
        value = maybe_function(x,y,a,1,1,gamma)
    return round(value,4)

In [5]:
def calc_os_for_multiple_attr(df):
    df['value']=df.apply(lambda row: abs(row[0]-0), axis =1)
    columns=len(df.columns)
    j=0
    while j < columns-2:
        df['value']=df.apply(lambda row: classification(row['value'], abs(row[j+1]-0), 0.5, 1, 1, 0.5),
                                                    axis = 1)
        j += 1
    return df

In [21]:
data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/data/more_attr_without_target.csv', sep=';')

In [22]:
data

,x,y,z,w
0,0.00,0.00,0.00,0.00
1,1.00,0.00,0.00,0.00
2,0.00,0.00,0.00,1.00
3,0.50,0.50,0.50,0.50
4,1.00,0.00,1.00,0.00
5,0.00,1.00,0.00,1.00
6,0.80,0.80,0.50,0.20
7,0.00,1.00,1.00,1.00
8,0.75,0.27,0.80,0.25
9,0.58,0.96,0.80,0.50


In [23]:
vysl = calc_os_for_multiple_attr(data)

In [24]:
df_prepare = vysl.drop(columns=['value'])
df_sorted_asc =df_prepare.values
df_sorted_asc.sort()
final_sorted_asc=pd.DataFrame(df_sorted_asc, df_prepare.index, df_prepare.columns)

In [25]:
vysl_asc = calc_os_for_multiple_attr(final_sorted_asc)

In [26]:
vysl['value_asc'] = vysl_asc['value']

In [27]:
cols = list(final_sorted_asc.drop(columns=['value']).columns)
cols.reverse()
final_df_desc=final_sorted_asc[cols]

In [28]:
vysl_desc = calc_os_for_multiple_attr(final_df_desc)

In [29]:
vysl['value_desc'] = vysl_desc['value']

In [30]:
vysl['result']=vysl.apply(lambda row: round((row['value_asc']+row['value_desc'])/2,2), axis=1)

In [31]:
vysl

,x,y,z,w,value,value_asc,value_desc,result
0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,1.00,0.00,0.00,0.00,0.00,0.50,0.00,0.25
2,0.00,0.00,0.00,1.00,0.50,0.50,0.00,0.25
3,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.50
4,1.00,0.00,1.00,0.00,0.50,1.00,0.00,0.50
5,0.00,1.00,0.00,1.00,0.50,1.00,0.00,0.50
6,0.80,0.80,0.50,0.20,0.70,0.80,0.70,0.75
7,0.00,1.00,1.00,1.00,1.00,1.00,0.50,0.75
8,0.75,0.27,0.80,0.25,0.57,0.57,0.52,0.54
9,0.58,0.96,0.80,0.50,1.00,1.00,1.00,1.00


In [ ]:
vysl.to_csv('/content/drive/MyDrive/Colab Notebooks/data/more_attr_example.csv')